In [1]:
import gymnasium as gym
import random

env = gym.make("room_env:RoomEnv-v2")
(obs, question), info = env.reset()
while True:
    action_qa = question[0]
    action_explore = random.choice(["north", "east", "south", "west", "stay"])
    (obs, question), reward, done, truncated, info = env.step(("wall", action_explore))
    if done:
        break

/home/tk/.virtualenvs/human-memory/lib/python3.10/site-packages/gymnasium/envs/registration.py:481: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes']
  logger.warn(
/home/tk/.virtualenvs/human-memory/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:168: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.deprecation(
/home/tk/.virtualenvs/human-memory/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:181: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(
/home/tk/.virtualenvs/human-memory/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:127: UserWarning: WARN: The obs returned by the 

In [2]:
observations, question = env.get_observations_and_question()
observations, question

/home/tk/.virtualenvs/human-memory/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.get_observations_and_question to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_observations_and_question` for environment variables or `env.get_wrapper_attr('get_observations_and_question')` that will search the reminding wrappers.
  logger.warn(


([['bedroom', 'north', 'livingroom', 100],
  ['bedroom', 'east', 'wall', 100],
  ['bedroom', 'south', 'wall', 100],
  ['bedroom', 'west', 'wall', 100],
  ['bed', 'atlocation', 'bedroom', 100],
  ['michael', 'atlocation', 'bedroom', 100],
  ['vincent', 'atlocation', 'bedroom', 100],
  ['headset', 'atlocation', 'bedroom', 100],
  ['agent', 'atlocation', 'bedroom', 100]],
 ['table', '?', 'livingroom', 100])

In [3]:
env.observations, env.question

/home/tk/.virtualenvs/human-memory/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.observations to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.observations` for environment variables or `env.get_wrapper_attr('observations')` that will search the reminding wrappers.
  logger.warn(
/home/tk/.virtualenvs/human-memory/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.question to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.question` for environment variables or `env.get_wrapper_attr('question')` that will search the reminding wrappers.
  logger.warn(


([['bedroom', 'north', 'livingroom', 100],
  ['bedroom', 'east', 'wall', 100],
  ['bedroom', 'south', 'wall', 100],
  ['bedroom', 'west', 'wall', 100],
  ['bed', 'atlocation', 'bedroom', 100],
  ['michael', 'atlocation', 'bedroom', 100],
  ['vincent', 'atlocation', 'bedroom', 100],
  ['headset', 'atlocation', 'bedroom', 100],
  ['agent', 'atlocation', 'bedroom', 100]],
 ['table', '?', 'livingroom', 100])

In [4]:
observations.append([1, 2, 3])
question[2] = "fuck"

observations, question

([['bedroom', 'north', 'livingroom', 100],
  ['bedroom', 'east', 'wall', 100],
  ['bedroom', 'south', 'wall', 100],
  ['bedroom', 'west', 'wall', 100],
  ['bed', 'atlocation', 'bedroom', 100],
  ['michael', 'atlocation', 'bedroom', 100],
  ['vincent', 'atlocation', 'bedroom', 100],
  ['headset', 'atlocation', 'bedroom', 100],
  ['agent', 'atlocation', 'bedroom', 100],
  [1, 2, 3]],
 ['table', '?', 'fuck', 100])

In [5]:
env.observations, env.question

([['bedroom', 'north', 'livingroom', 100],
  ['bedroom', 'east', 'wall', 100],
  ['bedroom', 'south', 'wall', 100],
  ['bedroom', 'west', 'wall', 100],
  ['bed', 'atlocation', 'bedroom', 100],
  ['michael', 'atlocation', 'bedroom', 100],
  ['vincent', 'atlocation', 'bedroom', 100],
  ['headset', 'atlocation', 'bedroom', 100],
  ['agent', 'atlocation', 'bedroom', 100]],
 ['table', '?', 'livingroom', 100])

In [39]:
class Foo:
    def __init__(self, arg) -> None:
        self.arg = arg

    def get_arg(self):
        return self.arg


foo = Foo({"arg": "foo"})
bar = foo.get_arg()
foo.arg = {"arg": "bar"}

foo.arg, bar

({'arg': 'bar'}, {'arg': 'foo'})

In [26]:
foo = ["foo"]
bar = {"answers": foo}

bar["answers"].append("bar")


foo, bar

(['foo', 'bar'], {'answers': ['foo', 'bar']})

In [ ]:
from room_env.envs.room2 import (
    Object,
    StaticObject,
    IndepdentObject,
    DependentObject,
    Room,
    RoomEnv2,
)

env = RoomEnv2(**config)
(obs, question), info = env.reset()

rewards = []
while True:
    (obs, question), reward, done, info = env.step("wall", "north")
    print(obs, question, reward, done, info)
    rewards.append(reward)
    if done:
        break

In [ ]:
from room_env.envs.room2 import (
    Object,
    StaticObject,
    IndepdentObject,
    DependentObject,
    Room,
    RoomEnv2,
)

env = RoomEnv2(**config)
(obs, question), info = env.reset()

obs, question

In [ ]:
rewards = []
while True:
    (obs, question), reward, done, info = env.step("wall", "north")
    print(obs, question, reward, done, info)
    rewards.append(reward)
    if done:
        break

In [ ]:
len(rewards)

In [ ]:
env.question

In [ ]:
env._get_hidden_global_state()

In [ ]:
(obs, question), reward, done, info = env.step("wall", "north")
(obs, question), reward, done, info